# 9.8 Funtion Calling

### function calliing - gpt3, gpt4만의 특징. 오직 이 두 모델에서만 사용 가능
#####   이 기능으로 우리가 할 수 있는것은 두가지다
#####   첫번째는 모델이 우리의 코드를 호출하도록 할 수 있고,
#####   아니면 모델이 우리가 원하느 ㄴ특정 모양과 형식의 output을 갖도록 강제할 수도 있다.

In [45]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.1)
prompt = PromptTemplate.from_template("How is the weather in {city}")

chain = prompt | llm
chain.invoke({"city": "Seoul"})

AIMessage(content='The weather in Seoul can vary depending on the time of year. In general, Seoul has four distinct seasons: spring, summer, fall, and winter.\n\nSpring (March to May) is mild and pleasant, with cherry blossoms blooming throughout the city.\n\nSummer (June to August) is hot and humid, with temperatures often reaching above 30 degrees Celsius. It can also be rainy during the summer months.\n\nFall (September to November) is cool and crisp, with beautiful foliage colors in the surrounding mountains.\n\nWinter (December to February) is cold and snowy, with temperatures often dropping below freezing. It can also be quite dry during the winter months.\n\nOverall, Seoul experiences a range of weather conditions throughout the year, so it is important to check the forecast before visiting.')

이렇게 코드를 실행하면 AI는 실시간 데이터를 가져올 수 없기 때문에 그냥 일반적인 그 지역의 날씨를 얘기해주게 된다.

그러나 만약 실시간 데이터를 가져올 수 있는 함수가 있다면 어떻게 될까??

함수에 필요한 정보는 아마 도시의 경도와 위도 정보와 같은 장소 정보뿐일 것이다.

아래 예시를 보면서 이해해보자.

In [46]:
def get_weather(lon, lat):
    print("call an api...")

이렇게 만약에 경도와 위도를 받아서 api를 호출하는 함수가 있다고 가정해보자. 지금 함수는 실제 동작은 안하지만 가정만 해보는 것이다.

우리가 함수 호출(function calling)으로 할 수 있는 일은 위와 같은 실망스러운 응답을 받는 대신에, gpt-3, gpt-4에게 우리에게 어떠어떠한 일을 해주는 get_weather이라는 함수가 있다고 말해주는 것이다.

그다음, gpt가 하는 일은 우리의 질문을 보게 될 것이고, 함수를 사용하는것이 이런 실망스러운 답변을 주는것 보다 나은지 판단할 것이다.

그럼 어떻게 만들 수 있을까?? 어떻게 GPT나 LLM을 그렇게 만들 수 있을까?? 어떻게 LLM이 get_weather이라는 함수가 있다는걸 알도록 할까?

정답은 매우 쉽다.

우리가 해야 하는건 스키마(schema)를 만드는 것이다. 이 함수를 위한 JSON Schema를 말이다.

다음 코드로 익혀보자.

In [47]:
weather_json = {
    "name": "get_weather",
    "description": "function that takes longitude and altitude to find the weather of a place",
    "parameters": {
        "type": "object",
        "properties":{
            "lon":{
                "type":"string",
                "description":"The longitude coordinate"
            },
            "lat":{
                "type":"string",
                "description":"The latitude coordinate"
            }
        }
    },
    "required":["lon", "lat"],
}

이렇게 JSON 형태로 함수에 필요한 데이터와 함수의 형태를 설명해주면 된다.

이제 이 함수를 ChatOpenAI가 함수를 사용할 수 있도록 어떻게 넣을까? 

우리는 langchain을 사용하고 있기 때문에 그렇게 할 수 있는 구체적인 방법이 있다.

다음과 같이 ChatOpenAI 클래스를 만들때 .bind()를 추가해주면 된다.

In [ ]:
llm = ChatOpenAI(
    temperature=0.1
).bind(
    function_call="auto",
        # 이 function call파라미터에서 우리가 할 수 있는 일은 기본적으로 모델이 특정 함수를 사용하도록 강제하는것이고,
        # 또는 모델이 함수를 사용해서 답변하거나 그냥 답변을 할 수 있도록 모델 스스로 선택하게 할 수 있다.
        # function_call={"name":"get_weather"}리고 하면 모델이 강제로 함수를 사용하도록 할 수 있다.
    functions=[
        weather_json # 여기에는 여러개의 JSON을 넣어 줄 수 있다.
    ]
)

chain = prompt | llm
response = chain.invoke({"city": "Seoul"})
print(response)

content='' additional_kwargs={'function_call': {'name': 'get_weather', 'arguments': '{"lon":"126.9784","lat":"37.5665"}'}}


출력을 보면 content는 비어있고, 'function call'이라고 함수를 호출해서 사용한 것을 할 수 있다.

In [49]:
response = response.additional_kwargs["function_call"]["arguments"]

In [50]:
import json

r = json.loads(response)
print(r)

{'lon': '126.9784', 'lat': '37.5665'}

이렇게 하면 실제로 사용할 수 있는 파이썬 객체를 만들 수 있다.

In [ ]:
get_weather(r['lon'], r['lat'])

우리가 한 일은 단지 함수의 모양, 형태만 설명한 것 뿐이다.

그다음에 "자, 이것이 우리의 함수가 작동하는 방식이야 그러니 properties의 형태로 줘"(weather_json의 properties형태를 말함).

보다시피 함수가 실제로 존재하지 않더라도 모델을 속여 원하는 형식으로 제공하게 하는 것은 아주 좋은 방법이다.

이는 올바른 형식의 질문을 하나의 코어로 생성 하려는 경우 사용할 수 있다.

왜냐면 prompt에서 모든 document들을 LLM에게 전달해줄 수 있기 때문이고, 또한 위의 weather_json처럼 스키마를 만들 수 있기 때문이다.

다음 코드는 QuizGPT를 위해 function calling을 사용한 코드이다.

In [51]:
function = {
    "name": "create_quiz",
    "description": "function that takes a list of questions and answers and returns a quiz",
    "parameters": {
        "type": "object",
        "properties": {
            "questions": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "question": {
                            "type": "string",
                        },
                        "answers": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "answer": {
                                        "type": "string",
                                    },
                                    "correct": {
                                        "type": "boolean",
                                    },
                                },
                                "required": ["answer", "correct"],
                            },
                        },
                    },
                    "required": ["question", "answers"],
                },
            }
        },
        "required": ["questions"],
    },
}

llm = ChatOpenAI(
    temperature=0.1,
).bind(
    function_call={
        "name": "create_quiz",
    },
    functions=[
        function,
    ],
)

prompt = PromptTemplate.from_template("Make a quiz about {city}")

chain = prompt | llm

response = chain.invoke({"city": "rome"})


response = response.additional_kwargs["function_call"]["arguments"]

response

'{"questions":[{"question":"What year was Rome founded?","answers":[{"answer":"753 BC","correct":true},{"answer":"476 AD","correct":false},{"answer":"100 AD","correct":false}]},{"question":"Who was the first emperor of Rome?","answers":[{"answer":"Julius Caesar","correct":false},{"answer":"Augustus","correct":true},{"answer":"Nero","correct":false}]},{"question":"What famous structure in Rome was built by the ancient Romans?","answers":[{"answer":"Eiffel Tower","correct":false},{"answer":"Colosseum","correct":true},{"answer":"Big Ben","correct":false}]}]}'

In [52]:
import json

for question in json.loads(response)["questions"]:
    print(question)

{'question': 'What year was Rome founded?', 'answers': [{'answer': '753 BC', 'correct': True}, {'answer': '476 AD', 'correct': False}, {'answer': '100 AD', 'correct': False}]}
{'question': 'Who was the first emperor of Rome?', 'answers': [{'answer': 'Julius Caesar', 'correct': False}, {'answer': 'Augustus', 'correct': True}, {'answer': 'Nero', 'correct': False}]}
{'question': 'What famous structure in Rome was built by the ancient Romans?', 'answers': [{'answer': 'Eiffel Tower', 'correct': False}, {'answer': 'Colosseum', 'correct': True}, {'answer': 'Big Ben', 'correct': False}]}
